<a href="https://colab.research.google.com/github/educatorsRlearners/hugging_face_course/blob/main/04_sharing_models_and_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using pretrained models

The Model Hub makes it incredibly straightforward to selecet a pretrained model based on the task we want to achieve. 

Why is that important? 

Becasue, as Alan Maley once said, 

> The question isn't "We have this tech; what can we use it for?" The question is, "We have this problem. What tech can we use to solve it?" 

So, if we want to fill some masks in French, we can import ```camembert-base``` and use it in a pipeline.

In [1]:
pip install transformers sentencepiece 

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 44.6 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 55.5 MB/s 
     |████████████████████████████████| 3.3 MB 55.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import  pipeline

checkpoint = 'camembert-base'
task = "fill-mask"

camembert_fill_mask = pipeline(task, checkpoint)

result = camembert_fill_mask("Le camembert est <mask> :) ")

for r in result:
  print(round(r['score'], 2), r["token_str"], r['sequence'], sep="---" )

0.49---délicieux---Le camembert est délicieux :)
0.11---excellent---Le camembert est excellent :)
0.03---succulent---Le camembert est succulent :)
0.03---meilleur---Le camembert est meilleur :)
0.03---parfait---Le camembert est parfait :)


Easy right? 

Just be careful to check and see that the checkpoint's head is suitable for the task. For instance, the ```camembert-base``` checkpoint has not been trained for sequence classification tasks. 

Finally, we can instantiate the checkpoint directly using the either the model architecture:

In [ ]:
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained(checkpoint)
moedl = CamembertForMaskedLM.from_pretrained(checkpoint)

But, if you want to be more flexible, then use the ```Auto*``` classes like this: 

In [ ]:
from transformers import  AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

# [Sharing pretrained models](https://huggingface.co/course/chapter4/3?fw=pt)

In short, there are three ways to create a new model in the model_hub:

- ```push_to_hub``` API
- ```huggingface_hub``` Python library
- web interface

The Python library as it is a wrapper for git while ```push_to_hub``` automatically generates a Model card with completed data. 


## ```push_to_hub``` API

To use this feature, we need to import a token like so: 

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Now, it is simply a matter of setting ```push_to_hub=True``` in the training arguments like this:

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc", save_strategy="epoch", push_to_hub=True
)

Now, every time we save the model (i.e., every epoch), it gets pushed to the hub.

When we're finished with training, the best practice is to then call ```.push_to_hub('message')``` on ```trainer```:

In [ ]:
trainer.push_to_hub("Finished training")